# 0. Import libraries and initialize random generator

In [1]:
#Python libraries
import numpy as np
from numpy.random import Generator, PCG64
from scipy import constants as sp
import time
import copy
import os

In [2]:
cwd = os.getcwd()
sys.path.append(cwd+'/modules')

In [3]:
#My libraries
from modules.generate_quantum_states import generate_M_random_DMs_file, generate_random_DMs_files_negativitywise 

En la celda a continuación vemos como funciona el generador de números aleatorios que he importado. Es un generador de la librería numpy.random que tiene un periodo de $2^{128}$. (numpy.org/doc/stable/reference/random/bit_generators/pcg64.html#numpy.random.PCG64). Para lo que necesitamos es más que suficiente. Por ejemplo, supongamos que vamos a generar 10^4 estados aleatorios. De acuerdo con el algoritmo que a continuación damos, necesitaremos generar 8 números aleatorios para cada estado, así pues, habremos de generar $8\times 10^4$ números aleatorios, que es muchos ordenes de magnitud menor que $2^{128}$.

In [ ]:
rg = Generator(PCG64(int(time.time())))
print(rg.random())
print(rg.random(4,))

# 1. Pure states generation

## 1.1. Theory and algorithm

Para un espacio, $\mathcal{H}$, de dimensión $N$, podemos tomar una base $\{|u_k\rangle ,\;\; k=1,...,N\equiv\text{dim}(\mathcal{H})\}$, de modo que $\forall|\psi\rangle\in\mathcal{H}$ se tiene:

>\begin{equation}
|\psi\rangle = \sum _{i=1}^{N} a_i|u_i\rangle \;\;\;\text{con}\;\;\;\sum |a_i|^2 = 1 \;\;\; a_i\in\mathbb{C}
\end{equation}

En general $a_j = r_j e^{i\phi _j}$ con $r_j>0$ y $\phi_j\in[0,2\pi)$. Para llevar a cabo la generación de estados aleatorios computacionalmente, vamos a considerar $0\leq r_j\leq 1$. Hemos de notar que considerar una cota superior para los módulos, $r_j$, no resta generalidad en tanto que es necesario normalizar el estado. Esto es, considerando $0\leq r_j\leq 1$ podemos samplear todo el espacio (salvo los estados separables, luego discutiremos porqué). Por ejemplo, podremos samplear un estado en que la amplitud asociada a uno de los estados base, $|u_k\rangle$ sea mucho mayor que la asociada al resto de estados. Para ello, simplemente tendrá que darse que para todos los estados $|u_j\rangle$ con $j\neq k$ se tenga $0\leq r_j<<r_k\leq 1$, de modo que al normalizar el estado por $\sqrt{\langle \psi|\psi\rangle}=\sqrt{\sum r_i ^2}$ se tendrá la relación de amplitudes deseada. De igual modo, podremos samplear estados en los que las amplitudes asociadas a todos los estados base sean muy parecidas, lo cual se tendrá cuando $r_j\simeq r_i$ $\forall i,j=1,...,N$.


Por otro lado, un estado separable se tendrá cuando $r_j=0$ $\forall j\neq k$. Puesto que para que se den este tipo de estados es necesario tener el 'cero' perfecto para los módulos de los coeficientes que acompañan a todos los estados base de $\mathcal{H}$ salvo a uno de ellos, podemos imaginarnos dicho estado como un punto (una variedad sin dimensión) sumergido en el espacio $\mathcal{H}$. Puesto que vamos a generar los módulos, $r_j$, y las fases, $\phi _j$, haciendo uso de un generador (numérico) de números aleatorios, nunca vamos a samplear el cero perfecto, por lo que realmente nunca vamos a tener un estado separable. Esto no va a ser un problema para nosotros. Cuando necesite estados separables los tomaré de los ficheros de estados que ya tengo o generaré otros nuevos haciendo uso de un algoritmo que considere implícitamente este detalle, i.e. que tome módulo cero para todos los estados base salvo para uno de ellos.

En el contexto del razonamiento que estábamos realizando hace dos párrafos, puedes comprobar que un estado $|\psi\rangle$ así construido (y normalizado) da lugar a una matriz densidad:

>\begin{equation}
|\psi\rangle\langle\psi| \doteq \frac{1}{\sum _i r_i ^2} \sum _{j,k} \tilde{c_{jk}} |u_j\rangle\langle u_k| \;\;\;\text{con}\;\;\; \tilde{c_{jk}} = r_jr_ke^{i(\phi _j -\phi _k})
\end{equation}

Así pues, la representación matricial de $|\psi\rangle\langle\psi|$ viene dada por:

>\begin{equation}
|\psi\rangle\langle\psi| \doteq \big(c_{jk}\big)_{j,k=1,...,N} \;\;\; \text{con}\;\;\; c_{jk} = \frac{\tilde{c_{jk}}}{\sum _i r_i ^2} = \frac{r_jr_ke^{i(\phi_j -\phi_k)}}{\sum _i r_i ^2}
\end{equation}

Esto es, $c_{jk}$ es el elemento de matriz que ocupa la fila $j$-ésima y la columna $k$-ésima. Hemos tomado implícitamente la base

>\begin{equation}
|u_k\rangle = \big(\delta _{ik}\big)_{i=1,...,N}
\end{equation}

Esto es, $|u_1\rangle^{T}=(1,0,...,0)$.

Así pues, para generar un estado cuántico aleatorio de $\mathcal{H}$ con $\text{dim}(\mathcal{H})\equiv N$, basta con:

1) Generar 4 números aleatorios $0\leq r_i\leq 1$, $i=1,...,N$.

2) Generar 4 números aleatorios $0\leq \phi_i< 2\pi$, $i=1,...,N$.

3) Construir la representación matricial $\big(c_{jk}\big)_{j,k=1,...,N}$ de acuerdo con la definición anterior.

## 1.2. Generation

### 1.2.1. Generate a single data file with random DMs regardless their negativities

In [4]:
current_working_directory = os.getcwd()
print(current_working_directory)

/home/julio/Documents/jupyterenvironment/code/entanglement_2x2_merge


In [5]:
filepath = current_working_directory+'/input_data/generated/pure_states/all_negativities.txt'
N=4
M=10000
rng = Generator(PCG64(int(time.time())))

In [12]:
%%time
negativity_mean, negativity_std = generate_M_random_DMs_file(N, M, filepath, rng, pure=True)

CPU times: user 2.67 s, sys: 63.9 ms, total: 2.73 s
Wall time: 2.73 s


In [13]:
print(negativity_mean, negativity_std)

0.2838676036133982 0.12376961455727861


### 1.2.2. Generate a collection of data files with random PURE DMs according to their negativities

# WARNING
Calling generate_random_DMs_files_negativitywise() is not general for whichever N. This is due to the fact that in the process we are using the function partial_transpose() which is, for the moment, only applicable to N=4.

In [8]:
current_working_directory = os.getcwd()
print(current_working_directory)

/home/julio/Documents/jupyterenvironment/code/entanglement_2x2_merge


In [9]:
neg_resolution = 0.1
M = 10000
N = 4; #WARNING: FOR THE MOMENT THIS VALUE CANNOT BE OTHER THAN 4
filepath_root = current_working_directory+'/input_data/generated/pure_states/negativity'
rng = Generator(PCG64(int(time.time())))

In [10]:
negativities_mean, negativities_std = generate_random_DMs_files_negativitywise(neg_resolution, M, N, filepath_root, rng, max_neg = 0.5, pure=True)

In [11]:
print(negativities_mean); print('\n'); print(negativities_std)

[0.06281419 0.15333697 0.25141403 0.35005161 0.44461528]


[0.02511362 0.02876996 0.02899423 0.02875178 0.02742743]


# 2. Mixed states generation

## 2.1. Theory and algorithm

Once we are able to generate random pure density matrices it is straightforward to generate mixed ones, just by taking convex combinations of the generated random pure density matrices. We shall first fix some integer L>1, so that a mixed states is a linear convex combination

>\begin{equation}
\rho = \sum _{i=1}^L c_i|\psi _i\rangle\langle\psi _i| \;\;\;\;\;\; \sum_{i=1}^L c_i = 1
\end{equation}

In order to generate such random mixed states I will fix some integer $L_{max}$, so that I iteratively generate random integers $L$ with $2<L<L_{max}$. For a given integer, $L$, I generate $L$ random numbers $0\leq \tilde{c_i}\leq1$ and $L$ random pure states, $|\psi _i\rangle\langle\psi _i|$. Then, I build the previous convex linear combination, where:

>\begin{equation}
c_i = \frac{\tilde{c_i}}{\sum _{j=1}^L \tilde{c_j}} \;\;\;\forall i
\end{equation}

## 2.2. Generation

### 2.2.1. Generate a single data file with random DMs regardless their negativities

In [4]:
current_working_directory = os.getcwd()
print(current_working_directory)

/home/julio/Documents/jupyterenvironment/code/entanglement_2x2_merge


In [5]:
filepath = current_working_directory+'/input_data/generated/mixed_states/all_negativities.txt'
N=4
M=10000
rng = Generator(PCG64(int(time.time())))

In [6]:
%%time
negativity_mean, negativity_std, L_mean, L_std, purity_mean, purity_std = generate_M_random_DMs_file(N, M, filepath, rng, pure=False, L_min=2, L_max=10)

CPU times: user 8.56 s, sys: 76.2 ms, total: 8.64 s
Wall time: 8.7 s


In [7]:
print(negativity_mean, negativity_std); print('\n'); print(L_mean, L_std); print('\n'); print(purity_mean, purity_std)

0.05341298478436195 0.07828753329168717


5.4953 2.2982989165902685


(0.47096831757914565+0j) 0.1358556402767338


### 2.2.2. Generate a collection of data files with random MIXED DMs according to their negativities

# WARNING
Calling generate_random_DMs_files_negativitywise() is not general for whichever N. This is due to the fact that in the process we are using the function partial_transpose() which is, for the moment, only applicable to N=4.

In [15]:
current_working_directory = os.getcwd()
print(current_working_directory)

/home/julio/Documents/jupyterenvironment/code/entanglement_2x2_merge


In [19]:
neg_resolution = 0.1
M = 100
N = 4; #WARNING: FOR THE MOMENT THIS VALUE CANNOT BE OTHER THAN 4
filepath_root = current_working_directory+'/input_data/generated/mixed_states/trial'
rng = Generator(PCG64(int(time.time())))

In [20]:
%%time
negativities_mean, negativities_std, Ls_mean, Ls_std, purities_mean, purities_std = generate_random_DMs_files_negativitywise(neg_resolution, M, N, filepath_root, rng, max_neg = 0.5, pure=False, L_min=3, L_max=7)

CPU times: user 2min 13s, sys: 16 ms, total: 2min 14s
Wall time: 2min 13s


In [21]:
print(negativities_mean); print('\n'); print(negativities_std); print('\n')
print(Ls_mean); print('\n'); print(Ls_std); print('\n')
print(purities_mean); print('\n'); print(purities_std); print('\n')

[0.02991114 0.1444349  0.23576945 0.32827351 0.41789382]


[0.0327916  0.02782789 0.02556131 0.02347584 0.01464519]


[4.67 4.01 3.56 3.23 3.05]


[1.03975959 1.00493781 0.79145436 0.52640289 0.29580399]


[0.45413701 0.56697033 0.6638168  0.77479489 0.89049377]


[0.07175508 0.07312907 0.07913669 0.06921745 0.03722914]




As L grows, the purity decays!

In order to generate these folders with M=10^4, I expect the previous cell to take a runtime of approximately 15 min.

# 99. Debugging cells

In [58]:
a = np.zeros((4,), dtype = complex)
print(a)
onerandom = 2*sp.pi*rg.random()
print(onerandom)
a[0] = 3*np.exp(1.j*onerandom)
print(a)

[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
6.185007390365431
[2.98555325-0.29406081j 0.        +0.j         0.        +0.j
 0.        +0.j        ]


In [71]:
N=4
rng = Generator(PCG64(int(time.time())))
modules = rng.random((N,))
print(modules)
phases = rng.random((N,))
#rng.random() returns a random number in (0,1). In order to get one random number in (0,2\pi) we must multiply
#rng.random() by 2\pi. The scipy library implements scientific constants. Namely, scipy.constants.pi gives \pi.
phases = 2*sp.pi*phases
print(phases)

[0.45693748 0.90025702 0.65952452 0.68247266]
[4.63950717 3.13703934 3.55092786 4.02470988]


In [3]:
rng = Generator(PCG64(int(time.time())))
current_working_directory = os.getcwd()

In [11]:
random_int(57,61,rng)

59

In [20]:
a = rng.random((3,))
print(a)
a = a/np.sum(a)
print(a)

[0.84724284 0.15943537 0.53801098]
[0.54848758 0.10321518 0.34829724]


In [50]:
generate_random_pure_density_matrix(4,rng)

array([[ 0.44801229+0.j        , -0.40296899-0.03555404j,
         0.16304707-0.19209632j,  0.10412028-0.09655469j],
       [-0.40296899+0.03555404j,  0.36527591+0.j        ,
        -0.13140959+0.18572223j, -0.08598946+0.09510999j],
       [ 0.16304707+0.19209632j, -0.13140959-0.18572223j,
         0.14170447+0.j        ,  0.07929315+0.00950457j],
       [ 0.10412028+0.09655469j, -0.08598946-0.09510999j,
         0.07929315-0.00950457j,  0.04500734+0.j        ]])

In [118]:
a = np.ones((4,), dtype=complex)
print(a)
print(a[1].imag)

[1.+0.j 1.+0.j 1.+0.j 1.+0.j]
0.0


In [18]:
a = np.zeros((2,2), dtype=complex)
print(a)

[[0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]


In [22]:
a = np.array([[1,1], [2,2]])
b = np.array([[1,2], [3,4]])
print(a[:,:]+6*b[:,:])

[[ 7 13]
 [20 26]]


In [16]:
generate_random_mixed_density_matrix(4, 3, rng)

array([[ 0.20316817+0.j        , -0.07598519+0.10219242j,
         0.11401406+0.01027428j, -0.08378962+0.07587625j],
       [-0.07598519-0.10219242j,  0.82161717+0.j        ,
         0.2815922 -0.00174493j,  0.26964013+0.19270863j],
       [ 0.11401406-0.01027428j,  0.2815922 +0.00174493j,
         0.64292009+0.j        ,  0.12807525+0.31861297j],
       [-0.08378962-0.07587625j,  0.26964013-0.19270863j,
         0.12807525-0.31861297j,  0.33229457+0.j        ]])

In [46]:
True*True*False*True*True

0

In [43]:
one_DM = np.array([[ 0.44801229+0.j        , -0.40296899-0.03555404j,
         0.16304707-0.19209632j,  0.10412028-0.09655469j],
       [-0.40296899+0.03555404j,  0.36527591+0.j        ,
        -0.13140959+0.18572223j, -0.08598946+0.09510999j],
       [ 0.16304707+0.19209632j, -0.13140959-0.18572223j,
         0.14170447+0.j        ,  0.07929315+0.00950457j],
       [ 0.10412028+0.09655469j, -0.08598946-0.09510999j,
         0.07929315-0.00950457j,  0.04500734+0.j        ]])    

In [4]:
a = np.array([[1,2],[3,4]])
print(np.mean(a))

2.5
